In [1]:
import json
import pickle
#import numpy as np
#import seaborn as sns
#import matplotlib.pyplot as plt

#from itertools import chain
#from collections import Counter
#from sklearn.metrics import confusion_matrix
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.metrics import classification_report
from sklearn.metrics import precision_score, recall_score, f1_score

In [21]:
with open("/Utilisateurs/umushtaq/emotion_analysis_comics/ddialogue/saved_models/dd_utt_Llama-3.2-1B-Instruct-bnb-4bit/ddialogue_results_2.pickle", 'rb') as f:
    
    results = pickle.load(f)

In [22]:
grounds = results['grounds']
predictions = results['predictions']

In [23]:
len(grounds), len(predictions)

(1000, 1000)

In [24]:
grounds = [json.loads(x)["emotions"] for x in grounds]

In [25]:
predictions = [json.loads(x["content"]) for x in predictions]

In [26]:
predictions = [x['emotions'] for x in predictions]

In [27]:
len(grounds), len(predictions)

(1000, 1000)

In [28]:
bad_idx = []

for i, (l_g, l_p) in enumerate(zip(grounds, predictions)):
    if len(l_g) != len(l_p):
        print(i)
        bad_idx.append(i)

214
369
659


In [29]:
grounds = [item for i, item in enumerate(grounds) if i not in bad_idx]
predictions = [item for i, item in enumerate(predictions) if i not in bad_idx]

In [30]:
grounds = [item for sublist in grounds for item in sublist]
predictions = [item for sublist in predictions for item in sublist]

In [31]:
print(classification_report(grounds, predictions, digits=3))

              precision    recall  f1-score   support

       anger      0.629     0.333     0.436       117
     disgust      0.750     0.273     0.400        44
        fear      0.500     0.294     0.370        17
   happiness      0.678     0.551     0.608      1015
     neutral      0.898     0.946     0.921      6269
     sadness      0.472     0.245     0.323       102
    surprise      0.509     0.483     0.496       116

    accuracy                          0.863      7680
   macro avg      0.634     0.446     0.508      7680
weighted avg      0.851     0.863     0.854      7680



In [32]:
filtered = [(g, p) for g, p in zip(grounds, predictions) if g != "neutral"]

# Separate the filtered ground truths and predictions
filtered_grounds, filtered_predictions = zip(*filtered) if filtered else ([], [])

In [33]:
if filtered_grounds:
    micro_f1 = f1_score(filtered_grounds, filtered_predictions, average="micro")
    print("Micro F1-score:", micro_f1)
else:
    print("No non-neutral classes to evaluate.")

Micro F1-score: 0.49326718639262934


In [34]:
emotion_map = {
    0: "neutral",
    1: "anger",
    2: "disgust",
    3: "fear",
    4: "happiness",
    5: "sadness",
    6: "surprise"
}

In [35]:
reverse_emotion_map = {v: k for k, v in emotion_map.items()}

In [36]:
grounds = [reverse_emotion_map[emotion] for emotion in grounds]
predictions = [reverse_emotion_map[emotion] for emotion in predictions]

In [37]:
neutral_idxs = [idx for idx, elem in enumerate(grounds) if elem == 0]

In [38]:
filtered_y_true = [x for idx, x in enumerate(grounds) if idx not in neutral_idxs]
filtered_y_pred = [x for idx, x in enumerate(predictions) if idx not in neutral_idxs]

In [39]:
from sklearn.metrics import f1_score
import numpy as np

# Example ground truth (y_true) and predicted labels (y_pred)
#y_true = np.array([0, 1, 1, 2, 2, 2, 0, 1, 2, 0])  # True labels
#y_pred = np.array([0, 1, 0, 2, 2, 1, 0, 0, 2, 2])  # Predicted labels

# Class to exclude
#excluded_class = 0

# Mask to filter out the excluded class
#mask = grounds != excluded_class

# Filtered labels
#filtered_y_true = grounds[mask]
#filtered_y_pred = predictions[mask]

#filtered_y_true = [x for x in grounds if x != excluded_class]
#filtered_y_pred = [x for x in predictions if x != excluded_class]

# Compute F1-scores without the excluded class
micro_f1 = f1_score(filtered_y_true, filtered_y_pred, average='micro')
weighted_f1 = f1_score(filtered_y_true, filtered_y_pred, average='weighted')

# Print results
print(f"Micro F1-score (excluding class ): {micro_f1:.4f}")
print(f"Weighted F1-score (excluding class ): {weighted_f1:.4f}")


Micro F1-score (excluding class ): 0.4933
Weighted F1-score (excluding class ): 0.6420
